<a href="https://colab.research.google.com/github/lookinsight/ml/blob/main/20221115_ML_LightGBM_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LightGBM

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# https://www.kaggle.com/datasets/kartik2112/fraud-detection
file_url = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/fraud.csv'
df = pd.read_csv(file_url)

In [ ]:
df.head()

In [ ]:
df.columns

* trans_date_trans_time : 거래 시간
* cc_num : 카드 번호. 고윳값이기 때문에 여기서는 id처럼 활용할 수 있음
* merchant : 거래 상점
* category : 거래 상점의 범주(애완용품, 여행, 엔터테인먼트 등)
* amt: 거래금액 (amount)
* first / last : 이름
* gender : 성별
* street / state / zip : 고객 거주지 정보
* lat / long : 고객주소에 대한 위도 및 경도
* city_pop : 고객의 zipcode 속하는 인구 수
* job : 직업
* dob : 생년월일
* trans_num : 거래번호
* unix_time : 거래시간 (유닉스 타임스탬프)
* merch_lat / merch_long : 상점의 위경도
---
* is_fraud : 사기거래 여부 (이상거래 여부) -> 종속변수

In [ ]:
df.info()

In [ ]:
df.info(show_counts=True)

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
df.describe()

## 전처리

In [ ]:
# 사용되지 않는 변수 제거
df.drop(['first', 'last', # 이름
         'street', 'city', 'state', 'zip', # 주소
         'trans_num', 'unix_time', # 거래번호 / 유닉스타임 (중복)
         'job', 'merchant' # 직업, 가게
         ], axis=1, inplace=True)

In [ ]:
# 날짜 형태의 데이터 Object 문자열로 저장되었다가 
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df.info()

## 피처엔지니어링

* 원래 고객의 거래패턴에서 벗어나는 거래

In [ ]:
#@title 결제 금액
# Z 점수 (정규분포) <- (x - 평균) / 표준편차
# amt_info = df.groupby('cc_num').agg(['mean', 'std'])
# cc_num : 카드번호별 그룹을 묶어서, agg -> 여러 그룹함수(여러개의 값들을 통해 계산하는 통계값)
# mean = 평균 / std = 표준편차 => amt(결제금액) => cc_num
amt_info = df.groupby('cc_num').agg(['mean', 'std']).amt.reset_index()
amt_info.head()

In [ ]:
# A.merge(B, on = KEY, how=WAY)
# A라는 데이터프레임에 B를 합쳐주겠다 => (index?) => cc_num
# LEFT
df = df.merge(amt_info, on = 'cc_num', how='left')

In [ ]:
df.columns 

In [ ]:
df[['cc_num', 'amt', 'mean', 'std']].head()

In [ ]:
# (x - 평균)/표준편차
df['amt_z_score'] = (df['amt'] - df['mean']) / df['std']

In [ ]:
df['amt_z_score'].head()

In [ ]:
df.drop(['mean', 'std'], axis=1, inplace=True)

In [ ]:
#@title 범주별 결제금액
# 결제를 한 사람의 카드번호 / 결제가 일어난 상점의 종류(분류)
# agg -> 그룹을 대상으로 통계값. 그룹함수
# mean : 평균 / std : 표준편차
category_info = df.groupby(['cc_num', 'category']).agg(['mean', 'std'])['amt'].reset_index()

In [ ]:
category_info.head()

In [ ]:
# cc_num, category -> merge. / mean, std => z_score / mean, std? drop
df = df.merge(category_info, on=['cc_num', 'category'], how='left')
df[['cc_num', 'category', 'amt', 'mean', 'std']].head()

In [ ]:
df['cat_z_score'] = (df['amt'] - df['mean']) / df['std']

In [ ]:
df['cat_z_score'].head()

In [ ]:
df.drop(['mean','std'], axis = 1, inplace = True)

In [ ]:
import geopy.distance

In [ ]:
# 두 지점 간의 거리 (위경도)
# geopy.distance.distance((lat1, lng1), (lat2, lng2))

In [ ]:
# coordinate 좌표 (위.경도)
# -- 좌표 : (위도, 경도) - latitude(위도)->북/남 / longitude(경도)->동/서
# 1. 상점의 위치 (merchant)
# 2. 고객의 위치 (customer)
df['merch_coord'] = pd.Series(zip(df.merch_lat, df.merch_long))
df['cust_coord'] = pd.Series(zip(df['lat'], df['long']))

In [ ]:
# df['distance'] = df.apply(lambda x: geopy.distance.distance(x['merch_coord'], x['cust_coord']), axis = 1)
df['distance'] = df.apply(lambda x: geopy.distance.distance(x['merch_coord'], x['cust_coord']).km, axis=1)

In [ ]:
df2 = df.copy()

In [ ]:
df2.distance

In [ ]:
# .km 빼먹으면 -> 단위를 포함해서 object -> object => float
# df['distance'] = df['distance'].str.split(expand=True)[0].astype('float64')

In [ ]:
distance_info = df.groupby('cc_num').agg(['mean', 'std'])['distance'].reset_index()

In [ ]:
distance_info

In [ ]:
df = df.merge(distance_info, on = 'cc_num', how='left')
df['distance_z_score'] = (df['distance'] - df['mean']) / df['std']

In [ ]:
df['distance_z_score']

In [ ]:
df.drop(['mean', 'std'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.isna().mean()

In [ ]:
df['dob'] # yyyy-MM-dd

In [ ]:
pd.to_datetime(df['dob'])

In [ ]:
# pd.to_datetime(df['dob']).dt # 날짜/시간 관련된 메소드.프로퍼티
pd.to_datetime(df['dob']).dt.year

In [ ]:
df['age'] = 2021 - pd.to_datetime(df['dob']).dt.year # 만 나이

In [ ]:
df.age

In [ ]:
df.drop(['cc_num', 'lat', 'long',
         'merch_lat', 'merch_long', 'dob',
         'merch_coord', 'cust_coord'], axis=1, inplace=True)

In [ ]:
df = pd.get_dummies(df, columns = ['category', 'gender'], drop_first=True)

In [ ]:
df # 과거의 데이터 -> 미래를 예측 / 과거데이터 (훈련셋) - 미래 데이터 (시험셋)
# trans_date_trans_time => 인덱스 -> 훈련셋 / 시험셋

In [ ]:
df.set_index('trans_date_trans_time', inplace=True)

In [ ]:
df.index # 2020-07-01

In [ ]:
train = df[df.index < '2020-07-01']
test = df[df.index >= '2020-07-01']

In [ ]:
X_train = train.drop('is_fraud', axis = 1) 
y_train = train['is_fraud'] 
X_test = test.drop('is_fraud', axis = 1) 
y_test = test['is_fraud'] 

In [ ]:
import lightgbm as lgb

In [ ]:
model = lgb.LGBMClassifier(random_state = 100) 
model.fit(X_train, y_train) 
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

In [ ]:
1 - df.is_fraud.mean()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

In [ ]:
def confusion_matrix_view(y_test, pred):
    cf_matrix = confusion_matrix(y_test, pred)
    print(cf_matrix)
    group_names = ['TN','FP','FN','TP']
    group_counts = ["{0:0.0f}".format(value) for value in
                    cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
                        cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
            zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='coolwarm')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.show()
    print(classification_report(y_test, pred))

cf_matrix = confusion_matrix(y_test, pred)
print(cf_matrix)
group_names = ['TN','FP','FN','TP']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='coolwarm')
plt.ylabel('True')
plt.xlabel('Predicted')
plt.show()

In [ ]:
# 정밀도(1종오류), 재현도(2종오류), f1-점수(전체)
print(classification_report(y_test, pred))

In [ ]:
# 0, 1 => 분류되기 전에 얼마나 가까운지? -> 0.5? 0.4? 0.8? (확률)
proba = model.predict_proba(X_test)
proba
# 각행 -> 입력받은 test별로
# 1열 : 0에 대한 예측값
# 2열 : 1에 대한 예측값 <= 이상거래 여부

In [ ]:
proba[:, 1] # 모든 행의 2열을 불러옴 -> 1에 대한 예측값 (이상거래 여부)

In [ ]:
proba = proba[:, 1]

In [ ]:
# default : 0.5 -> 크면은 1로 쳐주자
# case1 : 0.2 더 헐겁게 하자
# case2 : 0.8 엄격하게 하자
# T/F -> 1/0 -> int -> 1.0.10....
proba_int1 = (proba > 0.2).astype('int')
proba_int2 = (proba > 0.8).astype('int')

In [ ]:
confusion_matrix_view(y_test, pred)

In [ ]:
# 0.2 -> 1종오류는 늘고, 2종오류는 줄었어요
confusion_matrix_view(y_test, proba_int1)

In [ ]:
# 0.8 -> 2종오류는 늘고, 1종오류는 줄었어요
confusion_matrix_view(y_test, proba_int2)

# ROC 곡선 & AUC
* [참고](https://losskatsu.github.io/machine-learning/stat-roc-curve/#2-%EB%AF%BC%EA%B0%90%EB%8F%84%EC%99%80-%ED%8A%B9%EC%9D%B4%EB%8F%84)
* 이진분류 모델을 평가하는 방법으로 기준점에 영향을 받지 않기 때문에 여러 모델을 비교할 때 요긴하게 사용
* AUC는 ROC 곡선의 아래 면적을 의미
    * 0.5~1 사이의 값을 지니며 높을수록 좋은 모델

## ROC 곡선
> 민감도, 특이도 개념을 활용

### 민감도 (TPR)
$TPR = \frac{TP(참 양성)}{TP(참 양성) + FN(거짓 음성)}$
* 재현율과 수식이 같음
* 실제 1인 것 중 얼마만큼 제대로(1로) 예측되었는지
* 1에 가까울 수록 좋은 수치

### 특이도 (TNR) **
$TNR = \frac{TN(참 음성)}{FP(거짓 양성) + TN(참 음성)}$
* 실제 0인 것 중 얼마만큼 제대로(0로) 예측되었는지
* 1에 가까울 수록 좋은 수치
$FPR = \frac{FP(거짓 양성)}{FP(거짓 양성) + TN(참 음성)}$
* 실제 0인 것 중 얼마만큼 잘못(1로) 예측되었는지
* 0에 가까울 수록 좋은 수치
$FPR = 1 - TNR$

![ROC](https://i.imgur.com/euCumVh.png)
* 기준점을 바꿨을 때 TPR, FPR이 어떻게 바뀌는지 보여주는 그래프
* 최악의 경우 (학습이 전혀 안될 경우) -> 빨간색 점선

### AUC (Area Under the ROC Curve)
* ROC 곡선의 아래쪽에 해당하는 면적
* 0.5~1 사이의 값을 가지며 커질 수록 더 좋은 분류기라는 의미
![AUC](https://i.imgur.com/udlCMW4.png)

In [ ]:
# 0과 1 얼마나 잘 분리하는지 
roc_auc_score(y_test, proba)

## 랜덤 그리드 서치 

- 그리드 서치: 모든 조합에 대해서 모델링 <- 더 좋은? 
- 랜덤 그리드 서치: 더 넓은 영역의 하이퍼파라미터 값을 더 짧은 시간에 다양하게 활용 -> 일부만 추출해서 하기 때문에 <- 더 짧은 시간

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params = {
    'n_estimators' : [100, 500, 1000],
    'learning_rate' : [0.01, 0.05, 0.1, 0.3],
    'lambda_l1' : [0, 10, 20, 30, 50],
    'lambda_l2' : [0, 10, 20, 30, 50],
    'max_depth' : [5, 10, 15, 20],
    'subsample' : [0.6, 0.8, 1]
}

## L1 정규화(lambda_l1)와 L2 정규화(lambda_l2)
* 라쏘 회귀(lasso regression) - L1 정규화
* 릿지 회귀(rigde regression) - L2 정규화
> 둘 다 매개 변수에 패널티를 가해서 그 영향력(계수)을 감소 시키는 방법.<br>
오버피팅을 방지하는 목적으로 쓰임

In [ ]:
model2 = lgb.LGBMClassifier(random_state=100)
rs = RandomizedSearchCV(model2, param_distributions=params, n_iter=30,
                        scoring='roc_auc', random_state=100, n_jobs=-1)

In [ ]:
import time 
start = time.time()   # 시작시간 설정 
rs.fit(X_train, y_train) 
print(time.time() - start) 

In [ ]:
rs.best_params_

In [ ]:
rs_proba = rs.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, rs_proba[:, 1]) 

In [ ]:
rs_proba_int = (rs_proba[:, 1] > 0.2).astype('int')
confusion_matrix_view(y_test, rs_proba_int)

In [ ]:
confusion_matrix_view(y_test, proba_int1)

## train() 함수

* model.fit(X_train, y_train) => 학습 
* model.train <- fit.
---
||train|fit|
|-|-|-|
|검증셋|모델링 - 검증셋|검증 x|
|데이터셋||데이터프레임 -> 별도 포맷 변환|데이터프레임, 시리즈|
|하이퍼파라미터|하이퍼파라미터 Default x|기본값| 
|사이킷런 연동|X|O (그리드 서치...)|

학습 -> 학습셋 => model =>> 시험셋

학습 -> 학습셋과는 별개로 (검증셋) => model 

In [ ]:
# 훈련셋 / 시험셋 
# 훈련셋 / 검증셋 / 시험셋 
train = df[df.index < '2020-01-01']
val = df[(df.index >= '2020-01-01') & (df.index < '2020-07-01')] 
test = df[df.index >= '2020-07-01']

In [ ]:
def get_X_y(df):
    X = df.drop('is_fraud', axis = 1)
    y = df.is_fraud
    return (X,y) 

In [ ]:
# 훈련셋, 검증셋, 시험샛(X, y) ) 
X_trian, y_train = get_X_y(train) 
X_val, y_val = get_X_y(val)
X_test, y_test = get_X_y(test) 

In [ ]:
# 데이터셋 변환
d_train = lgb.Dataset(X_train, label=y_train)
d_val = lgb.Dataset(X_val, label=y_val)

In [ ]:
param_set = rs.best_params_
param_set['metrics'] = 'auc'

In [ ]:
param_set

In [ ]:
model_train = lgb.train(param_set, d_train, valid_sets=[d_val],
                        # early_stopping_rounds : 학습시간을 제한
                        # verbose_eval : 중간결과를 특정 간격으로 출력
                        early_stopping_rounds=100, verbose_eval=100)

In [ ]:
pred_train = model_train.predict(X_test)

In [ ]:
roc_auc_score(y_test, pred_train) 

In [ ]:
feature_imp = pd.DataFrame({'features': X_train.columns, 'values': model.feature_importances_}) 
plt.figure(figsize = (20, 10)) 
sns.barplot(x = 'values', y = 'features',
            data = feature_imp.sort_values(by = 'values', ascending = False).head(10))